Here we will explore collecting songs from spotify APIs and how to ask for what we want, that is:
- Once the user inputs a song, we will ask spotify for the features of the song
- Then we will match with another song with similar features, and recommend it to the user
The steps are:
- Find playlists in spotify with lots of songs
- Create a list with them
- Extract name of songa and audio features from each one
- Create a DF

Then:
- Cluster songs according to audio features

In this notebook I will work with playlists containing female artists only.

In [1]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="6582a8f807004727b3bc8a7c281ae02c",
    client_secret="c38fc3efa7f74def856cb96d000fd6ea"))

In [26]:
#create a list of playlists url's:

list_url = ["https://open.spotify.com/playlist/13qezNPeoOZbnpgYdvfXzw", "https://open.spotify.com/playlist/7vse1mpYM4ZZMAnFjtDmxn", "https://open.spotify.com/playlist/5BrB90Tlv372JYksHKEUUC", "https://open.spotify.com/playlist/4gGVp5hZBGMT5u8Umwy71i", "https://open.spotify.com/playlist/69IzgRxJudWrroRkFz6rqD", "https://open.spotify.com/playlist/7Bz2d7LtQAA4sdqjK54ihN", "https://open.spotify.com/playlist/3jN4ozUr5QnxLJAuWSOz8L", "https://open.spotify.com/playlist/7nb8QoKz6X7j43a8JLDWWa", "https://open.spotify.com/playlist/0Elk90H3u3Pii6HmU7uwDQ", "https://open.spotify.com/playlist/5jL36juHp2hI5p4w7Otkp8", "https://open.spotify.com/playlist/1FNxkddOTy0dEQjzO605Mu", "https://open.spotify.com/playlist/71AoD5gr7igctQEioLcrgs", "https://open.spotify.com/playlist/28PCyERc69PVABnVlhzCzK", "https://open.spotify.com/playlist/1AHXVPpZj0ttq2kEHfRrBz", "https://open.spotify.com/playlist/7GQpGQ7mo1qNgjRYzTIROw", "https://open.spotify.com/playlist/1n6ApCcBNm1LlObavhViZV", "https://open.spotify.com/playlist/52p0TeTGoMt2bdYaCgDgGi", "https://open.spotify.com/playlist/1vs8WUfA0QQO4V0rtVpFKX"]


doesn't have the "next"
(delete afterwards)

playlists = []
for i in list_url:
    playlists.append(sp.user_playlist_tracks("spotify", i))
print(len(playlists))

In [28]:
#with "next"
playlists = []
for i in list_url:
    track = sp.user_playlist_tracks("spotify", i)
    playlists.append(track)
    while track['next']:
        track = sp.next(track)
        playlists.append(track) 
print(len(playlists))


37


In [29]:
len(playlists[0]["items"])

100

In [75]:
playlists[0].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [78]:
playlists[0]['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [114]:
playlists[0]['items'][0]['track']['artists'][0]["name"]

'Evanescence'

In [30]:
## making if iterate trhough the list of playlists I have:
songs = []
for i in range(len(playlists)):
    for song in playlists[i]["items"]:
        songs.append(song["track"]["name"])
print(songs)

['Bring Me To Life', 'Like The Way I Do', 'The Chain - 2004 Remaster', 'Stars', 'White Rabbit', 'Total Eclipse of the Heart', "Your Daddy Don't Know", 'What About Love?', 'You Keep Me Hangin On', 'Time After Time', 'Call Me', "Don't Speak", "What's Up?", 'Crimson and Clover', 'Back on the Chain Gang - 2007 Remaster', 'Love Is A Battlefield', 'Celebrity Skin', 'Whatcha Do to My Body', 'Tell It to My Heart', 'Running Up That Hill (A Deal With God) - 2018 Remaster', 'Complicated', "Don't It Make Ya Feel", 'Close My Eyes Forever', 'High School Confidential', 'The Ballad Of Lucy Jordan', 'Big Yellow Taxi', 'Blackbird', 'Ironic - 2015 Remaster', 'What I Am', 'Me and Bobby McGee', 'I Just Want to Make Love to You', 'Something To Talk About', 'Barracuda', 'Edge of Seventeen - 2016 Remaster', "Call Me When You're Sober", 'A Whiter Shade of Pale', 'Luka', 'We Belong', 'Bette Davis Eyes', 'Fast Car', 'All I Wanna Do', 'Stupid Girl', 'Brass in Pocket - 2006 Remaster', 'Love Is', '99 Luftballons', 

In [91]:
len(songs)

2975

In [32]:
len(set(songs))

2618

In [ ]:
## now, get audio features of the songs:
##try with one song first: search for it, get irs uri, and use in the audio feature function

sp.search('High Hopes')["tracks"]["items"][0]["uri"]

In [90]:
##now try to iterate create a list with all of the uris
uris = []
for i in range(len(playlists)):
    for uri in playlists[i]["items"]:
        uris.append(uri["track"]["uri"])
print(len(uris))

2975


In [115]:
##now try to iterate create a list with all of the artists
artists = []
for i in range(len(playlists)):
    for artist in playlists[i]["items"]:
        artists.append(artist["track"]["artists"][0]["name"])
print(len(artists))

2975


In [34]:
## get audio features
audio_feat = []
for i in uris:
    audio_feat.append(sp.audio_features(i))
print(len(audio_feat))

2975


In [53]:
audio_feat[2]

[{'danceability': 0.545,
  'energy': 0.67,
  'key': 9,
  'loudness': -8.81,
  'mode': 1,
  'speechiness': 0.0496,
  'acousticness': 0.009,
  'instrumentalness': 0.000822,
  'liveness': 0.0451,
  'valence': 0.481,
  'tempo': 151.553,
  'type': 'audio_features',
  'id': '5e9TFTbltYBg2xThimr0rU',
  'uri': 'spotify:track:5e9TFTbltYBg2xThimr0rU',
  'track_href': 'https://api.spotify.com/v1/tracks/5e9TFTbltYBg2xThimr0rU',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5e9TFTbltYBg2xThimr0rU',
  'duration_ms': 270213,
  'time_signature': 4}]

In [ ]:
# now i have: a list with song names, and a list with their audio features. Lets put them in a df

In [88]:
# first, lets separate each feature, so the df has one column to each feature

danceability = []
for song in audio_feat:
    for i in song:
        danceability.append(i["danceability"])
len(danceability)

2975

In [87]:
loudness = []
for song in audio_feat:
    for i in song:
        loudness.append(i["loudness"])
len(loudness)

2975

In [86]:
speechiness = []
for song in audio_feat:
    for i in song:
        speechiness.append(i["speechiness"])
len(speechiness)

2975

In [85]:
acousticness = []
for song in audio_feat:
    for i in song:
        acousticness.append(i["acousticness"])
len(acousticness)

2975

In [84]:
energy = []
for song in audio_feat:
    for i in song:
        energy.append(i["energy"])
len(energy)

2975

In [83]:
instrumentalness = []
for song in audio_feat:
    for i in song:
        instrumentalness.append(i["instrumentalness"])
len(instrumentalness)

2975

In [82]:
tempo = []
for song in audio_feat:
    for i in song:
        tempo.append(i["tempo"])
len(tempo)

2975

In [116]:
import pandas as pd

songs_features_dict = {
    "name_of_song" : songs,
    "artist": artists,
    "danceability": danceability,
    "energy": energy,
    "loudness": loudness,
    "speechiness": speechiness,
    "acousticness": acousticness,
    "instrumentalness": instrumentalness,
    "tempo": tempo,
    "uri": uris
}

songs_features = pd.DataFrame(songs_features_dict)

In [99]:
len(songs) == len(danceability) == len(artists) == len(uris) == len(loudness) == len(speechiness) == len(acousticness) == len(instrumentalness) == len(tempo)

True

In [117]:
songs_features.head()

,name_of_song,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,uri
0,Bring Me To Life,Evanescence,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,spotify:track:0COqiPhxzoWICwFCS4eZcp
1,Like The Way I Do,Melissa Etheridge,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,spotify:track:5cj6toDqMzkbpgkGoRav0S
2,The Chain - 2004 Remaster,Fleetwood Mac,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,spotify:track:5e9TFTbltYBg2xThimr0rU
3,Stars,Grace Potter & The Nocturnals,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,spotify:track:6s7sq2Quh4TxHRfdk6eFxk
4,White Rabbit,Jefferson Airplane,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,spotify:track:4vpeKl0vMGdAXpZiQB2Dtd


In [119]:
songs_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2975 entries, 0 to 2974
Data columns (total 10 columns):
name_of_song        2975 non-null object
artist              2975 non-null object
danceability        2975 non-null float64
energy              2975 non-null float64
loudness            2975 non-null float64
speechiness         2975 non-null float64
acousticness        2975 non-null float64
instrumentalness    2975 non-null float64
tempo               2975 non-null float64
uri                 2975 non-null object
dtypes: float64(7), object(3)
memory usage: 232.5+ KB
